In [102]:
import pandas as pd
import datetime as dt
import altair as alt

In [103]:
# read in state data
tx_all = pd.DataFrame(pd.read_csv('data/texas_data.csv'))
wi_all = pd.DataFrame(pd.read_csv('data/wisconsin_data.csv'))

acs_tx = pd.read_csv('data/acs_tx.csv')
acs_wi = pd.read_csv('data/acs_wi.csv')

/var/folders/pr/r11ws70528760pvl1q2tychh0000gn/T/ipykernel_72143/171777722.py:2: DtypeWarning: Columns (3,4,5,6,8,9,10,11,12,13,14,18,19,20,21,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  tx_all = pd.DataFrame(pd.read_csv('data/texas_data.csv'))
/var/folders/pr/r11ws70528760pvl1q2tychh0000gn/T/ipykernel_72143/171777722.py:3: DtypeWarning: Columns (3,4,5,6,8,9,11,12,13,18,19,20,21,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  wi_all = pd.DataFrame(pd.read_csv('data/wisconsin_data.csv'))


In [104]:
# read in additional data
burglary = pd.read_csv('data/burglary.csv')

In [105]:
tx_all.columns

Index(['Unnamed: 0', 'geo_state', 'geo_county', 'race', 'ethnicity', 'sex',
       'state', 'zip', 'booking_date', 'booking_time', 'charge', 'ftp',
       'all_ftp', 'any_ftp', 'booking_id', 'raw_row_number', 'age', 'bond',
       'note', 'release_date', 'release_time', 'severity', 'release_type',
       'warrant_type', 'length_of_stay', 'fine', 'court', 'statute'],
      dtype='object')

In [106]:
# remove unwanted columns
tx_all = tx_all.drop(columns=['Unnamed: 0', 'state', 'statute', 'bond'])
wi_all = wi_all.drop(columns=['Unnamed: 0','state', 'statute', 'bond'])
# set zipcodes to strings
# remove .0 and convert to string
#tx_all['zip'] = tx_all['zip'].astype(str)
#wi_all['zip'] = wi_all['zip'].astype(str)
#tx_all['zip'] = tx_all['zip'].str.split('.')[0]
#wi_all['zip'] = wi_all['zip'].str.split('.')[0]
# set dates to dt
tx_all['booking_date'] = pd.to_datetime(tx_all['booking_date'])
tx_all['release_date'] = pd.to_datetime(tx_all['release_date'])
wi_all['booking_date'] = pd.to_datetime(wi_all['booking_date'])
wi_all['release_date'] = pd.to_datetime(wi_all['release_date'])
# set times to dt
#tx_all['booking_time'] = pd.to_datetime(tx_all['booking_time'])
#tx_all['release_time'] = pd.to_datetime(tx_all['release_time'])
#wi_all['booking_time'] = pd.to_datetime(wi_all['booking_time'])
#wi_all['release_time'] = pd.to_datetime(wi_all['release_time'])



## Figure 1

Per capita booking rates for failure to pay only, and any failure to pay 

In [107]:
# Texas Data
# create two dfs, one with counties with any all_ftp true and one with counties with any any_ftp true
tx_all_ftp = tx_all.groupby('geo_county').filter(lambda x: x['all_ftp'].any()).reset_index()
tx_any_ftp = tx_all.groupby('geo_county').filter(lambda x: x['any_ftp'].any()).reset_index()

# keep only 01-01-2005 to 12-31-2017 booking data
tx_all_ftp = tx_all_ftp[(tx_all_ftp['booking_date'] >= '2005-01-01') & (tx_all_ftp['booking_date'] <= '2017-12-31')]
tx_any_ftp = tx_any_ftp[(tx_any_ftp['booking_date'] >= '2005-01-01') & (tx_any_ftp['booking_date'] <= '2017-12-31')]

In [108]:
# find burgulary booking rates per year
burglary_tx = (acs_tx.groupby('year')
                .agg(pop=('pop', 'sum'))
                .reset_index()
                .merge(burglary, on='year')
                .assign(burglary=lambda x: 1e6 * x['burglary_tx'] / x['pop'])
                [['year', 'burglary']]
                .dropna())

# make sure year is a datetime object
burglary_tx['year'] = pd.to_datetime(burglary_tx['year'], format='%Y')

In [109]:
# create a count column
tx_all_ftp['count'] = 1

# get year from booking date
tx_all_ftp['year'] = tx_all_ftp['booking_date'].dt.year

# get total bookings and all_ftp bookings per year and county 
tx_per_cap_county = (tx_all_ftp
    .groupby(['geo_county', 'year'])
    .agg(total_bookings=('count', 'size'),
         all_ftp_bookings=('all_ftp', 'sum'))
    .reset_index())

# from the authors:
  # Sometimes bookings include small numbers of individuals from other years,
  # even if the rest of the data from those years are not available. Typically,
  # this is because their sentences are very long. Filter those years.
tx_per_cap_county = (tx_per_cap_county
    .groupby('geo_county')
    .filter(lambda x: x['total_bookings'].iloc[0] > 0.05 * x['total_bookings'].max()))

# merge with acs data
tx_per_cap_county_with_acs = (tx_per_cap_county
    .merge(acs_tx, on=['geo_county', 'year'], how='left')
    .drop('Unnamed: 0', axis=1))

# calculate per capita bookings by year by millions
tx_bookings_per_cap_by_year = (tx_per_cap_county_with_acs
    .groupby(['year'])
    .apply(lambda df: pd.Series({
        'total_pop_represented': df['pop'].sum(),
        'per_capita_bookings': 1e6 * df['total_bookings'].sum() / df['pop'].sum(),
        'per_capita_ftp_bookings': 1e6 * df['all_ftp_bookings'].sum() / df['pop'].sum()}))
    .reset_index())

# keep only two decimal places
tx_bookings_per_cap_by_year['per_capita_bookings'] = tx_bookings_per_cap_by_year['per_capita_bookings'].round(2)
tx_bookings_per_cap_by_year['failure to pay only'] = tx_bookings_per_cap_by_year['per_capita_ftp_bookings'].round(2)

# make sure year is a datetime object
tx_bookings_per_cap_by_year['year'] = pd.to_datetime(tx_bookings_per_cap_by_year['year'], format='%Y')

/var/folders/pr/r11ws70528760pvl1q2tychh0000gn/T/ipykernel_72143/713826435.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({


In [128]:
# combine ftp with burgulary data
tx_per_cap_by_year = tx_bookings_per_cap_by_year.merge(burglary_tx, on='year')
tx_per_cap_by_year['state'] = 'Texas'

,year,total_pop_represented,per_capita_bookings,per_capita_ftp_bookings,failure to pay only,burglary,state
0,2005-01-01,3671543.0,91855.93,3117.490385,3117.49,766.025770,Texas
1,2006-01-01,3671543.0,87916.99,2820.612478,2820.61,769.468394,Texas
2,2007-01-01,3689803.0,95946.86,3241.636478,3241.64,778.536769,Texas
3,2008-01-01,5412340.0,92620.38,2655.413370,2655.41,826.775485,Texas
4,2009-01-01,5341831.0,89055.42,2726.967588,2726.97,857.801082,Texas
5,2010-01-01,5897611.0,80217.23,2516.442675,2516.44,801.788721,Texas
6,2011-01-01,6013668.0,76163.33,2679.230047,2679.23,724.948108,Texas
7,2012-01-01,6066095.0,75714.44,2554.031877,2554.03,675.515474,Texas
8,2013-01-01,6349064.0,70294.77,2807.185437,2807.19,611.208394,Texas
9,2014-01-01,6627500.0,72609.73,3037.042625,3037.04,547.025216,Texas


In [120]:
# Wisconsin Data
# create two dfs, one with counties with any all_ftp true and one with counties with any any_ftp true
wi_all_ftp = wi_all.groupby('geo_county').filter(lambda x: x['all_ftp'].any()).reset_index()
wi_any_ftp = wi_all.groupby('geo_county').filter(lambda x: x['any_ftp'].any()).reset_index()

# keep only 01-01-2005 to 12-31-2017 booking data
wi_all_ftp = wi_all_ftp[(wi_all_ftp['booking_date'] >= '2005-01-01') & (wi_all_ftp['booking_date'] <= '2017-12-31')]
wi_any_ftp = wi_any_ftp[(wi_any_ftp['booking_date'] >= '2005-01-01') & (wi_any_ftp['booking_date'] <= '2017-12-31')]

In [121]:
burglary_wi = (acs_wi.groupby('year')
                .agg(pop=('pop', 'sum'))
                .reset_index()
                .merge(burglary, on='year')
                .assign(burglary=lambda x: 1e6 * x['burglary_wi'] / x['pop'])
                [['year', 'burglary']]
                .dropna())

# make sure year is a datetime object
burglary_wi['year'] = pd.to_datetime(burglary_wi['year'], format='%Y')

In [122]:
# create a count column
wi_all_ftp['count'] = 1

# get year from booking date
wi_all_ftp['year'] = wi_all_ftp['booking_date'].dt.year

# get total bookings and all_ftp bookings per year and county 
wi_per_cap_county = (wi_all_ftp
    .groupby(['geo_county', 'year'])
    .agg(total_bookings=('count', 'size'),
         all_ftp_bookings=('all_ftp', 'sum'))
    .reset_index())

# from the authors:
  # Sometimes bookings include small numbers of individuals from other years,
  # even if the rest of the data from those years are not available. Typically,
  # this is because their sentences are very long. Filter those years.
wi_per_cap_county = (wi_per_cap_county
    .groupby('geo_county')
    .filter(lambda x: x['total_bookings'].iloc[0] > 0.05 * x['total_bookings'].max()))

# merge with acs data
wi_per_cap_county_with_acs = (wi_per_cap_county
    .merge(acs_wi, on=['geo_county', 'year'], how='left')
    .drop('Unnamed: 0', axis=1))

# calculate per capita bookings by year by millions
wi_bookings_per_cap_by_year = (wi_per_cap_county_with_acs
    .groupby(['year'])
    .apply(lambda df: pd.Series({
        'total_pop_represented': df['pop'].sum(),
        'per_capita_bookings': 1e6 * df['total_bookings'].sum() / df['pop'].sum(),
        'per_capita_ftp_bookings': 1e6 * df['all_ftp_bookings'].sum() / df['pop'].sum()}))
    .reset_index())

# keep only two decimal places
wi_bookings_per_cap_by_year['per_capita_bookings'] = wi_bookings_per_cap_by_year['per_capita_bookings'].round(2)
wi_bookings_per_cap_by_year['failure to pay only'] = wi_bookings_per_cap_by_year['per_capita_ftp_bookings'].round(2)

# make sure year is a datetime object
wi_bookings_per_cap_by_year['year'] = pd.to_datetime(wi_bookings_per_cap_by_year['year'], format='%Y')

/var/folders/pr/r11ws70528760pvl1q2tychh0000gn/T/ipykernel_72143/635959324.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({


In [123]:
# combine ftp with burgulary data (long format)
wi_per_cap_by_year = wi_bookings_per_cap_by_year.merge(burglary_wi, on='year')
wi_per_cap_by_year['state'] = 'Wisconsin'


In [124]:
# combine tx and wi data
tx_wi = pd.concat([tx_per_cap_by_year, wi_per_cap_by_year])

In [129]:
# FIGURE 1
scale = alt.Scale(domain=['failure to pay only', 'burglary'], range=['orange', 'red'])
title={'text': ['Per capita booking rates by state (2005-2017)'],
       'subtitle': [''],
      'color': 'black'}
states_per_cap = alt.Chart(tx_wi).mark_line(point=True).transform_fold(
    fold=['failure to pay only', 'burglary'], 
    as_=['Booking type', 'value']
).encode(
    alt.X('year', axis=alt.Axis(grid=False)).title(None),
    alt.Y('value:Q').title('Annual bookings per million population'),
    color=alt.Color('Booking type:N', scale=scale)
).facet(
    facet=alt.Facet('state:N', title=None),
    title=title
)

states_per_cap

alt.FacetChart(...)